Created to plot the output of coherence_test.py. May be edited to plot other versions of similar output from other files.

In [ ]:
import datetime
import os
import pickle
import sys

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# sys.path.append('../')
sys.path.append(os.path.join(os.path.dirname(""), os.pardir, os.pardir))
import coherence_analysis.utils as f

The section of the Brady's hotspring data we used has some missing time strectches. The next section of code shows the lengths of continuous records and the
associated time of recording.

In [ ]:
CONTINUOUS_IND_RANGES = [[0, 2314], [2314, 2488], [2488, 2740]]
CONTINUOUS_TIME_RANGES = [
    [
        datetime.datetime(2016, 3, 13, 0, 0, 18),
        datetime.datetime(2016, 3, 14, 14, 34, 48),
    ],
    [
        datetime.datetime(2016, 3, 14, 16, 40, 42),
        datetime.datetime(2016, 3, 14, 19, 35, 12),
    ],
    [
        datetime.datetime(2016, 3, 14, 19, 49, 19),
        datetime.datetime(2016, 3, 14, 23, 59, 19),
    ],
]

In [ ]:
file = r"D:\CSM\Mines_Research\large_scale_coherence_results\qr_metadata_160313000018_160314235949.pkl"
# file = r"D:\CSM\Mines_Research\Repositories\Coherence_Analyses\data\coherence_test_results\qr_metadata_160313000018_160315235949.pkl"
with open(file, "rb") as f:
    metadata = pickle.load(f)

metadata.keys()
metadata["sub_window_length"]
metadata["sampling_rate"]
metadata["files"][-1]
# meta data int array
met = np.array([int(a) for a in metadata["files"]])

intervals = met[1:] - met[:-1]
# plt.hist(intervals, bins=100)
np.where(intervals == np.max(intervals[2889:]))
# intervals[2877:2882] # [2879]
# metadata['files'][4632+342:4632+348]  #[4626:4632]
metadata["files"][4626:4632]
# plt.plot(intervals[4632+345:4632+375])
# plt.yscale("log")
# intervals[4632+345]
# metadata['files'][4977]

In [ ]:
(
    datetime.datetime.strptime(metadata["files"][0], "%y%m%d%H%M%S")
    + datetime.timedelta(seconds=30)
    - datetime.datetime.strptime(metadata["files"][1], "%y%m%d%H%M%S")
)

In [ ]:
# metadata['files']
file_dates = [
    mdates.date2num(datetime.datetime.strptime(f, "%y%m%d%H%M%S"))
    for f in metadata["files"]
]
fig, ax = plt.subplots(figsize=(7, 10))
ax.plot(file_dates, "-o")
ax.yaxis_date()
fig.autofmt_xdate()
# ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))

Get the catalog

In [ ]:
test_mdates = [
    datetime.datetime(2016, 1, 1, 0, 0, 0),
    datetime.datetime(2016, 1, 1, 0, 1, 0),
]
mds = mdates.date2num(test_mdates)
minimum_interval = mds[1] - mds[0]

In [ ]:
dataframe1 = pd.read_excel(
    r"C:\Users\issah\Downloads\BradyHotspring\catalog\Brady_hotspring_catalog.xlsx"
)

# convert dataframe times to datetime
dates = [
    datetime.date(
        dataframe1.iloc[i]["Year"],
        dataframe1.iloc[i]["Mon"],
        dataframe1.iloc[i]["Day"],
    )
    for i in range(len(dataframe1))
]
times = [dataframe1.iloc[i]["Time"] for i in range(len(dataframe1))]
datetimes = [
    datetime.datetime.combine(dates[i], times[i])
    for i in range(len(dataframe1))
]
plot_times = mdates.date2num(datetimes)


# filter out events that are too close together
time_intervals = plot_times[1:] - plot_times[:-1]
plot_peaks = [
    dataframe1.iloc[i]["Peak CC"]
    for i in range(min(88, len(plot_times)))
    if time_intervals[i] > minimum_interval
]
plot_times = [
    plot_times[i]
    for i in range(min(88, len(plot_times)))
    if time_intervals[i] > minimum_interval
]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

im = ax.plot(plot_times, plot_peaks, "ro")

ax.set_title("Catalog of Brady Hotspring Earthquakes")
ax.xaxis_date()
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))

In [ ]:
vibe_dataframe = pd.read_csv(
    r"C:\Users\issah\Downloads\BradyHotspring\metadata\Vibroseis_segy_timingLog_PoroTomo_Mar2016_Phases_1234_wUTM_v4.txt"
)


def _get_vibe_datetime(row):
    try:
        date_str = row["SigRec_segy_filename"][-10:-4]
        date_str = date_str[:2] + ":" + date_str[2:4] + ":" + date_str[4:]
        date_str = row[" DAQ_Date"] + " " + date_str
        out = datetime.datetime.strptime(date_str, "%m/%d/%Y %H:%M:%S")
    except ValueError:
        date_str = row[" DAQ_Date"] + " " + row[" DAQ_Time"]
        out = datetime.datetime.strptime(date_str, "%m/%d/%Y %H:%M:%S")
    return out


vibe_datetimes = [
    _get_vibe_datetime(vibe_dataframe.iloc[i]) for i in range(869, 1682)
]
vibe_plottimes = mdates.date2num(vibe_datetimes)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

plot_points = np.ones(len(vibe_plottimes))

im = ax.plot(vibe_plottimes, plot_points, "ro")

ax.set_title("Vibroseis Data Acquisition Times")
ax.xaxis_date()
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))

In [ ]:
# file = r"D:\CSM\Mines_Research\Repositories\Coherence_Analyses\test_results\exact_detection_significance_160312000048.pkl"
# file = r"D:\CSM\Mines_Research\Repositories\Coherence_Analyses\test_results\qr_detection_significance_160312000048.pkl"
# file = r"D:\CSM\Mines_Research\Repositories\Coherence_Analyses\test_results\qr_detection_significance_160314083848.pkl"
# file = r"D:\CSM\Mines_Research\Repositories\Coherence_Analyses\test_results\exact_detection_significance_160314083848.pkl"
# file = r"D:\CSM\Mines_Research\large_scale_coherence_results\qr_detection_significance_160313000018_160314235949.pkl"
# file = r"D:\CSM\Mines_Research\large_scale_coherence_results\exact_detection_significance_160313000018_160314235949.pkl"
# file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_08_16_24\exact_detection_significance_160313000018_160314235949.pkl"
# file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_08_16_24\qr_detection_significance_160313000018_160314235949.pkl"
# file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_08_16_24\svd_detection_significance_160313000018_160314235949.pkl"
# file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_08_21_24\exact_detection_significance_160313000018_160314235949.pkl"
# file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_08_21_24\qr_detection_significance_160313000018_160314235949.pkl"
# file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_08_21_24\svd_detection_significance_160313000018_160314235949.pkl"

# file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_08_26_24\exact_detection_significance_160313000018_160314235949.pkl"
# file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_08_26_24\qr_detection_significance_160313000018_160314235949.pkl"
# file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_08_26_24\svd_detection_significance_160313000018_160314235949.pkl"

# file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_08_30_24\exact_detection_significance_160313000018_160314235949.pkl"

In [ ]:
file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_08_26_24\qr_detection_significance_160313000018_160314235949.pkl"
with open(file, "rb") as f:
    detection_significances_qr = pickle.load(f)
file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_08_26_24\svd_detection_significance_160313000018_160314235949.pkl"
with open(file, "rb") as f:
    detection_significances_svd = pickle.load(f)


# plt.figure(figsize=(12, 6))
# # plt.plot(detection_significances[:100])

# plt.imshow(detection_significances.T[1:], aspect="auto")
# plt.colorbar()
# # detection_significances.shape

freqs = np.linspace(
    0,
    int(metadata["sampling_rate"] / 2 * (2 / 5)),
    detection_significances_qr.shape[1],
)
freq_lims = [freqs[1], freqs[-1]]

In [ ]:
file = r"D:\CSM\Mines_Research\Repositories\Coherence_Analyses\data\coherence_test_results\qr_detection_significance_160313000018_160315235949.pkl"
with open(file, "rb") as f:
    detection_significances_qr = pickle.load(f)
file = r"D:\CSM\Mines_Research\Repositories\Coherence_Analyses\data\coherence_test_results\svd_detection_significance_160313000018_160315235949.pkl"
with open(file, "rb") as f:
    detection_significances_svd = pickle.load(f)


# plt.figure(figsize=(12, 6))
# # plt.plot(detection_significances[:100])

# plt.imshow(detection_significances.T[1:], aspect="auto")
# plt.colorbar()
# # detection_significances.shape

freqs = np.linspace(
    0,
    int(metadata["sampling_rate"] / 2 * (2 / 5)),
    detection_significances_qr.shape[1],
)
freq_lims = [freqs[1], freqs[-1]]

In [ ]:
plot_qr = detection_significances_qr[:, 35].flatten()
plot_svd = detection_significances_svd[:, 35].flatten()
# plot_qr = detection_significances_qr.flatten()
# plot_svd = detection_significances_svd.flatten()

plt.figure(figsize=(15, 6))
plt.subplot(121)
plt.hist(plot_qr, bins=100, label="qr")
plt.hist(plot_svd, bins=100, alpha=0.5, label="standard")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.subplot(122)
plt.hist(plot_qr, bins=100, label="qr")
plt.hist(plot_svd, bins=100, alpha=0.5, label="standard")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.yscale("log")

In [ ]:
detection_significances_qr = detection_significances_svd

In [ ]:
np.sum(np.where(detection_significances_qr[:, 35] > 0.5))
len(np.where(detection_significances_qr[:, 35] > 0.5)[0])

In [ ]:
len(np.where(detection_significances_svd[:, 35] > 0.6)[0])

In [ ]:
time_lims = mdates.date2num(
    [
        datetime.datetime(2016, 3, 13, 0, 0, 18),
        datetime.datetime(2016, 3, 14, 23, 59, 19),
    ]
)
time_lims = mdates.date2num(
    [
        datetime.datetime(2016, 3, 13, 0, 0, 18),
        datetime.datetime(2016, 3, 14, 14, 34, 48),
    ]
)


fig, ax = plt.subplots(figsize=(15, 6))
# log_det_sig = np.log(detection_significances_qr[:, 1:])
log_det_sig = detection_significances_qr[:, :300]
im = ax.imshow(
    log_det_sig.T,
    # extent=[time_lims[0], time_lims[-1], freq_lims[-1], freq_lims[0]],
    aspect="auto",
    vmax=np.percentile(log_det_sig.T, 99),
    vmin=np.percentile(log_det_sig.T, 10),
    cmap="cividis",
)
ax.set_title("Log Detection Significance")
# ax.xaxis.set_ticks(np.linspace(time_lims[0], time_lims[-1], 15))
fig.colorbar(im)
# ax.xaxis_date()
# fig.autofmt_xdate()
# ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))

In [ ]:
CONTINUOUS_IND_RANGES = [[0, 2314], [2314, 2488], [2488, 2740]]
CONTINUOUS_IND_RANGES = [[0, 2314], [2314, 2488], [2488, 2740]]
# CONTINUOUS_IND_RANGES = [[0,1157], [1157, 1244], [1244, 1370]]
CONTINUOUS_TIME_RANGES = [
    [
        datetime.datetime(2016, 3, 13, 0, 0, 18),
        datetime.datetime(2016, 3, 14, 14, 34, 48),
    ],
    [
        datetime.datetime(2016, 3, 14, 16, 40, 42),
        datetime.datetime(2016, 3, 14, 19, 35, 12),
    ],
    [
        datetime.datetime(2016, 3, 14, 19, 49, 19),
        datetime.datetime(2016, 3, 14, 23, 59, 19),
    ],
]
i = 0
for r in CONTINUOUS_IND_RANGES:
    time_lims = mdates.date2num(
        [CONTINUOUS_TIME_RANGES[i][0], CONTINUOUS_TIME_RANGES[i][1]]
    )
    fig, ax = plt.subplots(figsize=(15, 6))
    log_det_sig = np.log(detection_significances_qr[r[0] : r[1], 1:])
    im = ax.imshow(
        log_det_sig.T,
        extent=[time_lims[0], time_lims[-1], freq_lims[-1], freq_lims[0]],
        aspect="auto",
        vmax=np.percentile(log_det_sig.T, 97),
        vmin=np.percentile(log_det_sig.T, 10),
        cmap="cividis",
    )
    ax.set_title("Log Detection Significance")
    ax.xaxis.set_ticks(np.linspace(time_lims[0], time_lims[-1], 15))
    fig.colorbar(im)
    ax.xaxis_date()
    fig.autofmt_xdate()
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))
    i += 1

In [ ]:
mean_ds = np.min(detection_significances_qr, axis=1)
mean_rm_ds = detection_significances_qr - mean_ds[:, np.newaxis]
# mean_ds = np.median(mean_rm_ds, axis=0)
# mean_rm_ds = mean_rm_ds - mean_ds[np.newaxis]
CONTINUOUS_IND_RANGES = [[0, 2314], [2314, 2488], [2488, 2740]]
# CONTINUOUS_IND_RANGES = [[0,1157], [1157, 1244], [1244, 1370]]
CONTINUOUS_TIME_RANGES = [
    [
        datetime.datetime(2016, 3, 13, 0, 0, 18),
        datetime.datetime(2016, 3, 14, 14, 34, 48),
    ],
    [
        datetime.datetime(2016, 3, 14, 16, 40, 42),
        datetime.datetime(2016, 3, 14, 19, 35, 12),
    ],
    [
        datetime.datetime(2016, 3, 14, 19, 49, 19),
        datetime.datetime(2016, 3, 14, 23, 59, 19),
    ],
]
i = 0
for r in CONTINUOUS_IND_RANGES:
    time_lims = mdates.date2num(
        [CONTINUOUS_TIME_RANGES[i][0], CONTINUOUS_TIME_RANGES[i][1]]
    )
    fig, ax = plt.subplots(figsize=(15, 6))
    # log_det_sig = np.log(mean_rm_ds[r[0]:r[1],1:])
    log_det_sig = mean_rm_ds[r[0] : r[1], 1:]
    im = ax.imshow(
        log_det_sig.T,
        extent=[time_lims[0], time_lims[-1], freq_lims[-1], freq_lims[0]],
        aspect="auto",
        cmap="cividis",
        vmax=np.percentile(log_det_sig.T, 99),
        vmin=np.percentile(log_det_sig.T, 5),
    )  # , vmax=np.percentile(log_det_sig.T, 97)), vmin=np.percentile(log_det_sig.T, 1), cmap="cividis")
    ax.set_title("Log Detection Significance")
    ax.xaxis.set_ticks(np.linspace(time_lims[0], time_lims[-1], 15))
    fig.colorbar(im)
    ax.xaxis_date()
    fig.autofmt_xdate()
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))
    i += 1

In [ ]:
len(mean_ds)

In [ ]:
mdates.num2date(vibe_plottimes[-1])

In [ ]:
mean_ds = np.min(detection_significances_qr, axis=1)
mean_rm_ds = detection_significances_qr - mean_ds[:, np.newaxis]

In [ ]:
times[1955]
catalog_event_inds = [
    1262,
    1324,
    1328,
    1329,
    1331,
    1402,
    1403,
    1404,
    1405,
    1406,
    1407,
    1955,
    1958,
    2732,
]
small_events = [1605, 1610, 1611, 1661, 1676, 1677, 1679, 1680]
event_051 = [1681, 1699]
event_052 = [1610, 1611, 1711]
mdates.num2date(times[1610])
# convert to datetime

In [ ]:
[a for a in range(10) if a is not None]

In [ ]:
plt.figure(figsize=(18, 6))
plt.plot(freqs[5:], mean_rm_ds[1955][5:], "--o")
plt.plot(freqs[5:], mean_rm_ds[1958][5:], "--o")
# plt.plot(freqs[5:], mean_rm_ds[2732][5:], "r-o")
# plt.plot(freqs[5:], mean_rm_ds[1330][5:], "k-o")
plt.plot(freqs[5:], mean_rm_ds[1611][5:], "r-o")
plt.vlines(7.5, 0, 0.4, color="k", linestyle="--")

In [ ]:
plt.figure(figsize=(18, 6))
plt.plot(mean_rm_ds[1211][5:300], "r-o")

In [ ]:
freqs[80]

In [ ]:
plt.figure(figsize=(18, 6))
f_ind = 25  # frequency index to plot
mean_ds = np.median(mean_rm_ds, axis=0)
mean_rm_ds = detection_significances_qr - mean_ds[np.newaxis]
plt.plot(mean_rm_ds[:, f_ind], "r-o")

# Define the window size
window_size = 2

# Create a window of ones, normalized to sum to 1
window = np.ones(window_size) / float(window_size)

# Convolve the data with the window
moving_average = np.convolve(mean_rm_ds[:, f_ind], window, "valid")

plt.plot(moving_average, "b-o", alpha=0.5)

In [ ]:
# plt.figure(figsize=(18, 6))
# plt.plot(detection_significances[20][5:], "r-o")

In [ ]:
freqs[25:35]

In [ ]:
i = 0
threshold = 0.04
last_ind = CONTINUOUS_IND_RANGES[-1][1]
# average_detection = np.mean(detection_significances_qr[:last_ind, 25:35], axis=1)
average_detection = detection_significances_qr[:last_ind, 35] - np.median(
    detection_significances_qr[:last_ind], axis=1
)
average_detection = np.mean(
    detection_significances_qr[:last_ind, 25:45]
    - np.median(detection_significances_qr[25:45], axis=1)[np.newaxis],
    axis=1,
)

average_detection = average_detection - np.median(average_detection)
# average_detection = np.mean(mean_rm_ds[:last_ind,5:15], axis=1)

times = np.empty(average_detection.shape[0], dtype=object)
# np.linspace(CONTINUOUS_TIME_RANGES[0][0], CONTINUOUS_TIME_RANGES[0][-1], CONTINUOUS_IND_RANGES[0][  ]- CONTINUOUS_IND_RANGES[0][0])
for r in CONTINUOUS_IND_RANGES:
    time_lims = mdates.date2num(
        [CONTINUOUS_TIME_RANGES[i][0], CONTINUOUS_TIME_RANGES[i][1]]
    )
    times[r[0] : r[1]] = np.linspace(
        time_lims[0], time_lims[-1], min(len(average_detection), r[1]) - r[0]
    )
    i += 1

fig, ax = plt.subplots(figsize=(12, 6))
im = ax.plot(
    times,
    average_detection,
    "ko",
    linewidth=2,
    label="detection parameter",
)
im1 = ax.plot(
    plot_times, plot_peaks, "ro", alpha=0.5, label="microseismic events"
)
im2 = ax.plot(
    vibe_plottimes, plot_points, "b*", label="vibroseis data acquisition times"
)
ax.plot(times[1965], 0.4, "go")
ax.xaxis_date()
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylim(-0.07, 1.05)
plt.xlabel("Time", fontsize=15)
plt.ylabel("Detection parameter", fontsize=15)
plt.legend(fontsize=12)

fig, ax = plt.subplots(figsize=(12, 6))
# log_det_sig = np.log(detection_significances[r[0]:r[1],1:])

# plt.plot(np.linspace(time_lims[0], average_detection[r[0]:r[1]], time_lims[-1], r[1]-r[0]), color="black", linewidth=2)
im = ax.plot(
    times[average_detection > threshold],
    average_detection[average_detection > threshold] - threshold,
    "o",
    color="black",
    linewidth=2,
)
im1 = ax.plot(plot_times, plot_peaks, "ro", alpha=0.5)
im2 = ax.plot(vibe_plottimes, plot_points, "b*")
ax.set_title("Detection parameter", fontsize=12)
# ax.xaxis.set_ticks(np.linspace(time_lims[0], time_lims[-1], 15))
# fig.colorbar(im)
ax.xaxis_date()
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))

In [ ]:
i = 0
threshold = 0.06
last_ind = CONTINUOUS_IND_RANGES[-1][1]
# average_detection = np.mean(detection_significances_qr[:last_ind, 25:35], axis=1)
average_detection = detection_significances_qr[:last_ind, 35] - np.median(
    detection_significances_qr[:last_ind], axis=1
)
average_detection = np.mean(
    detection_significances_qr[:last_ind, 40:60], axis=1
) - np.median(detection_significances_qr[:last_ind], axis=1)

average_detection = average_detection - np.median(average_detection)
# average_detection = np.mean(mean_rm_ds[:last_ind,5:15], axis=1)

times = np.empty(average_detection.shape[0], dtype=object)
# np.linspace(CONTINUOUS_TIME_RANGES[0][0], CONTINUOUS_TIME_RANGES[0][-1], CONTINUOUS_IND_RANGES[0][  ]- CONTINUOUS_IND_RANGES[0][0])
for r in CONTINUOUS_IND_RANGES:
    time_lims = mdates.date2num(
        [CONTINUOUS_TIME_RANGES[i][0], CONTINUOUS_TIME_RANGES[i][1]]
    )
    times[r[0] : r[1]] = np.linspace(
        time_lims[0], time_lims[-1], min(len(average_detection), r[1]) - r[0]
    )
    i += 1

fig, ax = plt.subplots(figsize=(12, 6))
im = ax.plot(
    times[1600:],
    average_detection[1600:],
    "ko",
    linewidth=2,
    label="detection parameter",
)
im1 = ax.plot(
    plot_times, plot_peaks, "ro", alpha=0.5, label="microseismic events"
)
# im2 = ax.plot(vibe_plottimes, plot_points, "b*", label="vibroseis data acquisition times")
ax.plot(times[2709], 0.06, "go")
ax.xaxis_date()
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylim(-0.07, 1.05)
plt.xlabel("Time", fontsize=15)
plt.ylabel("Detection parameter", fontsize=15)
plt.legend(fontsize=12)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(times, average_detection, color="black", linewidth=2)
plt.plot(times[:120], average_detection[:120], color="red", linewidth=2)
plt.plot(
    times[885:1580], average_detection[885:1580], color="red", linewidth=2
)

In [ ]:
without_vibe = list(range(120, 886)) + list(
    range(1581, len(detection_significances_qr[:, 35]))
)
# detection_significances_qr[without_vibe, 35]
plot_qr = detection_significances_qr[:, 35]
plot_svd = detection_significances_svd[:, 35]
# plot_qr = detection_significances_qr.flatten()
# plot_svd = detection_significances_svd.flatten()

plt.figure(figsize=(15, 13))
plt.subplot(221)
plt.hist(plot_qr, bins=150, label="qr")
plt.hist(plot_svd, bins=150, alpha=0.5, label="standard")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.subplot(222)
plt.hist(plot_qr, bins=100, label="qr")
plt.hist(plot_svd, bins=100, alpha=0.5, label="standard")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.yscale("log")

plot_qr = detection_significances_qr[without_vibe, 35]
plot_svd = detection_significances_svd[without_vibe, 35]
plt.subplot(223)
plt.hist(plot_qr, bins=100, label="qr")
plt.hist(plot_svd, bins=100, alpha=0.5, label="standard")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.subplot(224)
plt.hist(plot_qr, bins=100, label="qr")
plt.hist(plot_svd, bins=100, alpha=0.5, label="standard")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.yscale("log")

In [ ]:
colors = ["#800000", "#FFD700", "#663399", "#000000", "#008B8B"]
# plt.figure(figsize=(12, 6))
# plt.subplot(211)
plot_qr = detection_significances_qr[:, 35]
plot_svd = detection_significances_svd[:, 35]
plt.hist(plot_qr, bins=150, alpha=0.8, label="qr", color=colors[3])
# plt.hist(plot_svd, bins=150, alpha=0.5, label="standard", color=colors[3])
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
# plt.legend(fontsize=12)
plt.xlabel("Detection parameter", fontsize=15)
plt.ylabel("Counts", fontsize=15)


# plot_qr = detection_significances_qr[without_vibe, 35]
# plot_svd = detection_significances_svd[without_vibe, 35]
# plt.hist(plot_qr, bins=150, label="qr", color=colors[0])
# plt.hist(plot_svd, bins=150, alpha=0.5, label="standard", color=colors[3])
# plt.xticks(fontsize=12)
# plt.yticks(fontsize=12)
# plt.legend(fontsize=12)
# plt.xlabel("Detection parameter", fontsize=15)
# plt.ylabel("Counts", fontsize=15)

In [ ]:
# file = r"D:\CSM\Mines_Research\Repositories\Coherence_Analyses\test_results\qr_eig_estimates_160314083848.pkl"
file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_08_21_24\qr_eig_estimatess_160313000018_160314235949.pkl"
with open(file, "rb") as f:
    eig_estimates = pickle.load(f)
plt.figure(figsize=(12, 6))
plt.imshow(
    eig_estimates,
    aspect="auto",
    vmax=np.percentile(eig_estimates, 95),
    vmin=np.percentile(eig_estimates, 5),
    cmap="cividis",
)
plt.colorbar()

In [ ]:
average_eig_estimates = np.mean(eig_estimates[11:15], axis=0)
eigs_times = np.empty(eig_estimates.shape[1], dtype=object)
eigs_per_detection = int(
    eig_estimates.shape[1] / detection_significances.shape[0]
)
i = 0
for r in CONTINUOUS_IND_RANGES:
    time_lims = mdates.date2num(
        [CONTINUOUS_TIME_RANGES[i][0], CONTINUOUS_TIME_RANGES[i][1]]
    )
    eigs_times[eigs_per_detection * r[0] : eigs_per_detection * r[1]] = (
        np.linspace(
            time_lims[0],
            time_lims[-1],
            (
                min(len(average_eig_estimates), eigs_per_detection * r[1])
                - eigs_per_detection * r[0]
            ),
        )
    )
    i += 1

# plt.plot(average_eig_estimates, color="black", linewidth=2)
norm_average_eig_estimates = average_eig_estimates / max(average_eig_estimates)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
im = ax.plot(
    eigs_times, norm_average_eig_estimates, color="black", linewidth=2
)
im1 = ax.plot(plot_times, plot_peaks, "ro", alpha=0.5)
ax.xaxis_date()
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))
# plt.figure(figsize=(12, 6))

fig, ax = plt.subplots(figsize=(12, 6))

im = ax.plot(
    eigs_times[norm_average_eig_estimates > 0.25],
    norm_average_eig_estimates[norm_average_eig_estimates > 0.25],
    "o",
    color="black",
    linewidth=2,
)
im1 = ax.plot(plot_times, plot_peaks, "ro", alpha=0.5)
ax.set_title("Detection Significance")
# ax.xaxis.set_ticks(np.linspace(time_lims[0], time_lims[-1], 15))
# fig.colorbar(im)
ax.xaxis_date()
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))

In [ ]:
re = eig_estimates.reshape((60, int(eig_estimates.shape[1] / 30), 30))
# re[0, 0] == eig_estimates[0,:30]
red = re / np.tile(np.sum(re, axis=2), (30, 1, 1)).transpose(1, 2, 0)
# red = re / np.sum(re, axis=2)
red = red.reshape(eig_estimates.shape)
# np.sum(re, axis=2).shape
# red = np.log(red)

In [ ]:
start_time = datetime.datetime(2016, 3, 13, 0, 0, 18)
end_time = datetime.datetime(2016, 3, 14, 23, 59, 49)
time_lims = mdates.date2num([start_time, end_time])

# freqs = np.linspace(0, int(metadata["sampling_rate"]/2 * (3/5)), 1/metadata["sub_window_length"])
freqs = np.linspace(
    0, int(metadata["sampling_rate"] / 2 * (3 / 5)), red.shape[0]
)
freq_lims = [freqs[1], freqs[-1]]

In [ ]:
last_ind = -1
# plt.figure(figsize=(12, 6))
fig, ax = plt.subplots(figsize=(15, 6))
im = ax.imshow(
    red[1:, :last_ind],
    extent=[time_lims[0], time_lims[-1], 60, freq_lims[0]],
    aspect="auto",
    vmax=np.percentile(red[1:, :last_ind], 80),
    vmin=np.percentile(red[1:, :last_ind], 1),
    cmap="cividis",
)
ax.xaxis.set_ticks(np.linspace(time_lims[0], time_lims[-1], 15))
fig.colorbar(im)
ax.xaxis_date()
fig.aut0ofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))
# gca().xaxis.set_major_formatter(...)
# plt.colorbar()

In [ ]:
plt.plot(red[0, :30], "-o")
# plt.plot(eig_estimates[0, :60])

In [ ]:
start_ch = 3100
nchannels = 2000
fsize = 12
tick_size = 10

file = r"D:\CSM\Mines_Research\Test_data\Brady Hotspring\PoroTomo_iDAS16043_160313013018.h5"
data, _ = f.loadBradyHShdf5(file, normalize="no")

file = r"D:\CSM\Mines_Research\Test_data\Brady Hotspring\PoroTomo_iDAS16043_160313013048.h5"
data2, _ = f.loadBradyHShdf5(file, normalize="no")

data = np.append(data, data2, axis=1)

file = r"D:\CSM\Mines_Research\Test_data\Brady Hotspring\PoroTomo_iDAS16043_160313013118.h5"
data2, _ = f.loadBradyHShdf5(file, normalize="no")

data = np.append(data, data2, axis=1)
# data = np.append(data,data3,axis=1)
samples_per_sec = 1000

In [ ]:
# v_min = -np.percentile(np.absolute(data[start_ch:nchannels+start_ch]),95)
# v_max = np.percentile(np.absolute(data[start_ch:nchannels+start_ch]),95)
v_min = -0.015
v_max = 0.015

fig2 = plt.figure()
# img2 = plt.imshow(data[start_ch:nchannels+start_ch], cmap="RdBu",
img2 = plt.imshow(
    data,
    cmap="RdBu",
    vmin=v_min,
    vmax=v_max,
    aspect="auto",
    interpolation="none",
    extent=(0, 90, 5000, 1),
)
#    extent=(0,len(data_noise[start_ch:nchannels+start_ch][0])/samples_per_sec, ch_start, ch_start + nchannels))
# extent=(mdates.date2num(np.datetime64(props['GPSTimeStamp'])),mdates.date2num(np.datetime64(props['GPSTimeStamp'])+np.timedelta64(60,'s')), distances[0],distances[-1]))
plt.xlabel("Time (seconds)", fontsize=fsize)
plt.ylabel("Channels", fontsize=fsize)
# plt.title('Background noise',fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=tick_size)